Below are parameters of the experiments reported in [KLNO24](https://eprint.iacr.org/2024/1972).

In [1]:
def largest_a_below_binom_sum(n, threshold_power):
    """
    Returns the largest a such that sum_{i=0}^a binomial(n, i) < 2^threshold_power

    Parameters:
    - n: total number of bits
    - threshold_power: the exponent k such that the threshold is 2^k

    Returns:
    - a: largest integer such that the cumulative binomial sum is less than 2^threshold_power
    """
    threshold = 2^threshold_power
    running_sum = 0
    a = 0

    while True:
        running_sum += binomial(n, a)
        if running_sum >= threshold:
            return a - 1
        a += 1

# Example usage:
assert largest_a_below_binom_sum(256, 128) == 29

In [12]:
from rok_estimator.rok_estimator import *


def RelationRPWithKJL(ratio = 18, height = 105, secpar=110):
    class JL:
        kjl = ratio
        njl = height
        mjl = kjl * njl
        alpha_inf = largest_a_below_binom_sum(njl,secpar)
        
        def get_delta(self):
            return 2^(-secpar)
    
        def get_right_bound(self):
            return sqrt(self.mjl * self.njl)
            
        def get_left_bound(self):
            return self.alpha_inf / (2 * sqrt(self.mjl))
    
        def get_right_bound_inf(self):
            return self.mjl
            
        def get_left_bound_inf(self):
            return self.alpha_inf / (2)
            

    # class HJL: 
    #     kjl = ratio
    #     njl = 120
    #     mjl = kjl * njl
    #     alpha_inf = 42 
    
    #     def get_delta(self):
    #         return 2^(-110)
    
    #     def get_right_bound(self):
    #         return sqrt(207)
            
    #     def get_left_bound(self):
    #         return sqrt(7)
    
    #     def get_right_bound_inf(self):
    #         return self.mjl
            
    #     def get_left_bound_inf(self):
    #         return self.alpha_inf / (2)

    class LJL:
        kjl = ratio
        njl = 256
        mjl = kjl * njl
        
    
        def get_delta(self):
            return 2^(-128)
    
        def get_right_bound(self):
            return sqrt(337)
            
        def get_left_bound(self):
            return sqrt(30)
    
        def get_right_bound_inf(self):
            return sqrt(337)
            
        def get_left_bound_inf(self):
            return sqrt(30)
    
    
    
    jl = LJL()
    
    
    
    
    # jl = JL()
    
    print("Limits of structure: ", jl.mjl)
    
    
    g_first_branch = None
    g_second_branch = None
    
    g_second_branch_ext_2_norm = None
    g_second_branch_ext_inf_norm = None
    
    class Relation_RP(Relation):
        def execute(self, op, **kwargs):
            match op:
                case "srp":
                    
                    return self.pi_srp(**kwargs)
                case "to-first":
                    return self.pi_to_first(**kwargs)
                case "join":
                    return self.pi_join(**kwargs)
                case "fold-ext":
                    return self.pi_fold_ext(**kwargs)
                case "norm-poly":
                    return self.pi_norm_poly()
                case "urp":
                    return self.pi_urp()
                case "usplit":
                    return self.pi_usplit(**kwargs)
                case _:
                    return super().execute(op, **kwargs)
                
        def pi_norm_poly(self):
            comm = (3 * ceil(log(self.wdim, 2)) + 3 * self.rep) * self.ring.size_Rq()  # TODO: Overestimating. Some of the communication are short elements. Assuming that wdim is a power of two for now. Even if not, this would be an upper bound of the communication cost.   
            # comm = (2 * self.rep) * self.ring.size_Rq() + (1 * ceil(log(self.wdim, 2)) + 3 * self.rep) * self.ring.log_q  # TODO: Overestimating. Some of the communication are short elements. Assuming that wdim is a power of two for now. Even if not, this would be an upper bound of the communication cost.   
            comm = (2 * self.rep) * self.ring.size_Rq() + 2 * (ceil(log(self.wdim, 2)) + 3 * self.rep) * self.ring.log_q  # TODO: Overestimating. Some of the communication are short elements. Assuming that wdim is a power of two for now. Even if not, this would be an upper bound of the communication cost.   
            snd_err = 3 * ceil(log(self.wdim, 2))/(2**(self.ring.log_q * self.ring.residue_deg)) # TODO: Assuming that wdim is a power of two for now. Even if not, this would be an upper bound of the soundness error.
            rel_params = {
                # "ring": self.ring,
                # "trivial": self.trivial,
                "op_name": "norm-poly",
                "n_compress": self.n_compress + 2,
                # "n_commit": self.n_commit,
                "n_rel": self.n_rel + 2,
                # "wdim": self.wdim,
                # "rep": self.rep,
                # "log_beta_wit_2": bound_log_canon_2_from_log_coeff_inf(self.ring,self.log_beta_wit_inf, dim=self.wdim * (self.rep + ell)), # Measured in Frobenius norm
                # "log_beta_wit_2": self.log_beta_wit_2, # Measured in max ell_2-norm over all columns
                # "log_beta_wit_inf": self.log_beta_wit_inf
                "comm" : comm,      
                "acc_comm" : self.acc_comm + comm,       
                "snd_err" : snd_err,
                "acc_snd_err" : self.acc_snd_err + snd_err,           
                "log_beta_ext_2_func" : lambda x, _ : self.log_beta_wit_2, # perfect extraction
                "log_beta_ext_inf_func" : lambda x, _ : oo # extraction of ell_2-norm is perfect, then bound ell_inf-norm by norm conversion
            }      
            return replace(self, **rel_params)

        def pi_fold_ext(self,extra_rels):
            comm = (self.rep * extra_rels + extra_rels * extra_rels) * self.ring.size_Rq()
        
            rel_params = {
                "op_name": "fold-ext",
                "n_compress": self.n_compress + 1 + extra_rels, # set to try as one round it's ok as initially we don't have this extra row
                "n_rel": self.n_rel +  1 + extra_rels,
                "rep": self.rep + extra_rels,
                "comm": comm,      
                "acc_comm": self.acc_comm + comm,       
            }      
            return replace(self, **rel_params)
                
        def pi_srp(self):
            global g_first_branch 
            rel_params = {
                "n_compress": self.n_commit + 1,
                "n_rel": self.n_rel + 1,
                "log_slack_2_func": lambda x: x,
                "log_slack_inf_func": lambda x: x, 
            }
    
            g_first_branch = replace(deepcopy(self), **rel_params)
            comm = (self.n_commit + 1) * self.ring.size_Rq()  # for the new comm
            snd_err = self.wdim * (1 / 2**(self.ring.log_q * self.ring.residue_deg) + jl.get_delta() * euler_phi(self.ring.f) / jl.njl)
            global g_second_branch_ext_2_norm
            global g_second_branch_ext_inf_norm
    
            def log_beta_ext_2_func(x, _):
                global g_second_branch_ext_2_norm
                return g_second_branch_ext_2_norm + log(1 / jl.get_left_bound(), 2)
                
            def log_beta_ext_inf_func(x, _):
                global g_second_branch_ext_inf_norm 
                return g_second_branch_ext_inf_norm + log(1 / jl.get_left_bound_inf(), 2)
                
            rel_params = {
                "op_name": "srp 2 rel",
                "n_compress": self.n_commit + 1,
                "n_rel": 1,
                "wdim": ZZ(ceil(self.wdim / jl.kjl * self.rep)),
                "rep": 1,
                "comm": comm,      
                "acc_comm" : self.acc_comm + comm,       
                "snd_err" : snd_err,
                "acc_snd_err" : self.acc_snd_err + snd_err,    
                "log_beta_wit_2": self.log_beta_wit_2 + log(jl.get_right_bound(), 2) + log(sqrt(self.rep),2),
                "log_beta_wit_inf":self.log_beta_wit_inf + log(jl.get_right_bound_inf(), 2) + log(sqrt(self.rep),2), # can we say sth about infinity - norm?
                "log_beta_ext_2_func": log_beta_ext_2_func, # we extract norm from the second branch (relax it)
                "log_beta_ext_inf_func" : log_beta_ext_inf_func,  # can we say sth about infinity - norm?
                "log_slack_2_func": lambda x: 0,
                "log_slack_inf_func": lambda x: 0,
            }     
            
            # We immediately switch to the second branch to estimate binding and serve as an extraction checkpoint. 
            return replace(self, **rel_params)

        def pi_urp(self):
            comm = self.rep * self.ring.size_Rq() + (jl.njl * self.rep*self.ring.log_q) * 3 # doing intermediate batching 3 times is enough for all tests we consider and selected q
            snd_err = jl.get_delta() 
            # 2^(-256) is upper-bouding due to python insufficient precision (jl.njl / (self.rep* 2**self.ring.log_q))**4  + (jl.njl / (2 * self.rep* 2**self.ring.log_q ** 2))**2 + (jl.njl / (4 * self.rep * 2**self.ring.log_q)** 4) # 
            rel_params = {
                "op_name": "uns-rp",
                "n_compress": self.n_compress + 1,
                "n_rel": self.n_rel + 1,
                "comm": comm,      
                "acc_comm" : self.acc_comm + comm,  
                "snd_err" : snd_err,
                "acc_snd_err" : self.acc_snd_err + snd_err,    
                "log_beta_ext_2_func": lambda x, _: self.log_beta_wit_2 + log(jl.get_right_bound() / jl.get_left_bound(), 2),
                "log_beta_ext_inf_func": lambda x, _: oo,  # can we say sth about infinity - norm?
                "log_slack_2_func": lambda x: 0,
                "log_slack_inf_func": lambda x: 0,
            }     
                
            
            # We immediately switch to the second branch to estimate binding and serve as an extraction checkpoint. 
            return replace(self, **rel_params)

        def pi_usplit(self, d):
            # candidate = self.pi_split(d)

            comm = self.rep * self.ring.size_Rq() * (d**2 - 1)
            
            rel_params = {
                "op_name": "u-split",
                "n_compress": self.n_compress + d,
                "n_rel": self.n_rel + d,
                "wdim": ZZ(self.wdim / d),
                "comm": comm,      
                "acc_comm" : self.acc_comm + comm,  
            }     
                
            
            # We immediately switch to the second branch to estimate binding and serve as an extraction checkpoint. 
            return replace(self, **rel_params)
    
    
        def pi_to_first(self):
            global g_second_branch 
            g_second_branch = deepcopy(self)
            
            rel_params = {
                "op_name": "    1 rel",
                "n_rel": 1,
                "log_beta_ext_2_func" : lambda x, _ : x, # perfect extraction
                "log_beta_ext_inf_func" : lambda x, _ : x, # perfect extraction
                "acc_comm": g_second_branch.acc_comm,
                "comm": 0,
                "snd_err" : g_second_branch.snd_err,
                "acc_snd_err" : g_second_branch.acc_snd_err,    
                # "log_slack_2_func" : lambda x: 0, # perfect extraction
                # "log_slack_inf_func" : lambda x: 0 # perfect extraction
            }      
            global g_first_branch
            return replace(g_first_branch, **rel_params)
            
        def pi_join(self):
            def log_beta_ext_2_func(x, _):
                global g_second_branch_ext_2_norm
                g_second_branch_ext_2_norm = x
                return x
            def log_beta_ext_inf_func(x, _):
                global g_second_branch_ext_inf_norm
                g_second_branch_ext_inf_norm = x
                return x
    
            global g_second_branch

            print(g_second_branch.wdim, self.wdim)
            assert g_second_branch.wdim <= self.wdim
            assert g_second_branch.rep == 1
            assert self.rep == 1

            comm = (self.n_rel + g_second_branch.n_rel) * self.ring.size_Rq()  # for the new comm
            
            rel_params = {
                "op_name": "join",
                "rep": 2,
                "n_rel": self.n_rel + g_second_branch.n_rel,
                "n_compress": self.n_compress + g_second_branch.n_compress,
                "comm" : comm,      
                "acc_comm" : self.acc_comm + comm,  
                "log_beta_wit_2": max(self.log_beta_wit_2, g_second_branch.log_beta_wit_2),
                "log_beta_wit_inf": max(self.log_beta_wit_inf, g_second_branch.log_beta_wit_inf),
                "log_beta_ext_2_func": log_beta_ext_2_func, # perfect extraction, but we keep track of what was there
                "log_beta_ext_inf_func" : log_beta_ext_inf_func, # perfect extraction, but we keep track of what was there
                "log_slack_2_func": lambda x: 0, # assume no slack
                "log_slack_inf_func": lambda x: 0,
            }      
            return replace(self, **rel_params)
    return Relation_RP
        

# I use binary challenges in the subring of degree 128
C = ChallengeSet(cardinality = 2^128, gamma_2 = 128, theta_2 = 256, gamma_inf = 128, theta_inf = 256)

In [13]:
# A0

ring_params = {
    "f": 256, # N = 128
    "log_beta_sis_2": 44.3,
    "log_q": 50,
}

rel_params = {
    "wdim": 2**19,
    "rep": 1,
    "log_beta_wit_inf": 10
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm-poly", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("fold", {}),
    ("join", {}),
    ("batch", {}), 
])

uloop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm-poly", {}), 
    ("batch", {}), 
    ("usplit", {"d": ops_params["d"]}), 
    ("batch", {}), 
    ("urp", {}),
    ("fold", {}),
    ("batch", {}), 
])

ops = loop



ops = loop(1) + loop(2) * 7 + uloop(2)*3  + [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(8, 106, 106))
sim.ring.C = C


sim.execute(ops)
print(sim.ring)
sim.extract()
sim.show()

Limits of structure:  2048
coeff 2-norm
65536 262144
131072 131072
65536 65536
32768 32768
16384 16384
8192 8192
4096 4096
2048 2048
Ring parameters:
    conductor f: 256, degree phi: 128, modulus q: 2^50, beta_sis_2: 2^44.3000000000000
    SIS module rank n_sis: 12, target SIS security: 128, resulting SIS security: 128
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 100
    |R_q| = 800.000 B, |R_q^(n_sis)| = 9.37500 KB
 
Challenge set parameters:
    cardinality: 340282366920938463463374607431768211456
    forward ell_2 expansion factor gamma_2: 2^7
    inverse ell_2 expansion factor theta_2: 2^8
    forward ell_inf expansion factor gamma_inf: 2^7
    inverse ell_inf expansion factor theta_inf: 2^8
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |     524288 |   1 |    ( 23 | 23/  0 )  

In [17]:
# B0

ring_params = {
    "f": 256, # N = 128
    "log_beta_sis_2": 44.3,
    "log_q": 50,
}

rel_params = {
    "wdim": 2**21,
    "rep": 1,
    "log_beta_wit_inf": 10
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm-poly", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("fold", {}),
    ("join", {}),
    ("batch", {}), 
])

uloop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm-poly", {}), 
    ("batch", {}), 
    ("usplit", {"d": ops_params["d"]}), 
    ("batch", {}), 
    ("urp", {}),
    ("fold", {}),
    ("batch", {}), 
])

ops = loop



ops = loop(1) + loop(2) * 9 + uloop(2)*3  + [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(8, 106, 106))
sim.ring.C = C


sim.execute(ops)
print(sim.ring)
sim.extract()
sim.show()

Limits of structure:  2048
coeff 2-norm
262144 1048576
524288 524288
262144 262144
131072 131072
65536 65536
32768 32768
16384 16384
8192 8192
4096 4096
2048 2048
Ring parameters:
    conductor f: 256, degree phi: 128, modulus q: 2^50, beta_sis_2: 2^44.3000000000000
    SIS module rank n_sis: 12, target SIS security: 128, resulting SIS security: 128
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 100
    |R_q| = 800.000 B, |R_q^(n_sis)| = 9.37500 KB
 
Challenge set parameters:
    cardinality: 340282366920938463463374607431768211456
    forward ell_2 expansion factor gamma_2: 2^7
    inverse ell_2 expansion factor theta_2: 2^8
    forward ell_inf expansion factor gamma_inf: 2^7
    inverse ell_inf expansion factor theta_inf: 2^0
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |    209715

In [18]:
# C0

ring_params = {
    "f": 256, # N = 128
    "log_beta_sis_2": 44.3,
    "log_q": 50,
}

rel_params = {
    "wdim": 2**23,
    "rep": 1,
    "log_beta_wit_inf": 10
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm-poly", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("fold", {}),
    ("join", {}),
    ("batch", {}), 
])

uloop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm-poly", {}), 
    ("batch", {}), 
    ("usplit", {"d": ops_params["d"]}), 
    ("batch", {}), 
    ("urp", {}),
    ("fold", {}),
    ("batch", {}), 
])

ops = loop



ops = loop(1) + loop(2) * 11 + uloop(2)*3  + [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(8, 106, 106))
sim.ring.C = C


sim.execute(ops)
print(sim.ring)
sim.extract()
sim.show()

Limits of structure:  2048
coeff 2-norm
1048576 4194304
2097152 2097152
1048576 1048576
524288 524288
262144 262144
131072 131072
65536 65536
32768 32768
16384 16384
8192 8192
4096 4096
2048 2048
Ring parameters:
    conductor f: 256, degree phi: 128, modulus q: 2^50, beta_sis_2: 2^44.3000000000000
    SIS module rank n_sis: 12, target SIS security: 128, resulting SIS security: 128
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 100
    |R_q| = 800.000 B, |R_q^(n_sis)| = 9.37500 KB
 
Challenge set parameters:
    cardinality: 340282366920938463463374607431768211456
    forward ell_2 expansion factor gamma_2: 2^7
    inverse ell_2 expansion factor theta_2: 2^8
    forward ell_inf expansion factor gamma_inf: 2^7
    inverse ell_inf expansion factor theta_inf: 2^0
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth 

In [19]:
# A1

ring_params = {
    "f": 512, # N = 256
    "log_beta_sis_2": 45.7,
    "log_q": 50,
}

rel_params = {
    "wdim": 2**18,
    "rep": 1,
    "log_beta_wit_inf": 10
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm-poly", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("fold", {}),
    ("join", {}),
    ("batch", {}), 
])

uloop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm-poly", {}), 
    ("batch", {}), 
    ("usplit", {"d": ops_params["d"]}), 
    ("batch", {}), 
    ("urp", {}),
    ("fold", {}),
    ("batch", {}), 
])

ops = loop



ops = loop(1) + loop(2) * 6 + uloop(2)*3  + [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(8, 106, 106))
sim.ring.C = C


sim.execute(ops)
print(sim.ring)
sim.extract()
sim.show()

Limits of structure:  2048
coeff 2-norm
32768 131072
65536 65536
32768 32768
16384 16384
8192 8192
4096 4096
2048 2048
Ring parameters:
    conductor f: 512, degree phi: 256, modulus q: 2^50, beta_sis_2: 2^45.7000000000000
    SIS module rank n_sis: 7, target SIS security: 128, resulting SIS security: 142
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 100
    |R_q| = 1.56250 KB, |R_q^(n_sis)| = 10.9375 KB
 
Challenge set parameters:
    cardinality: 340282366920938463463374607431768211456
    forward ell_2 expansion factor gamma_2: 2^7
    inverse ell_2 expansion factor theta_2: 2^8
    forward ell_inf expansion factor gamma_inf: 2^7
    inverse ell_inf expansion factor theta_inf: 2^0
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |     262144 |   1 |    ( 23 | 23/  0 )        |     ( 

In [20]:
# B1

ring_params = {
    "f": 512, # N = 256
    "log_beta_sis_2": 47.7,
    "log_q": 50,
}

rel_params = {
    "wdim": 2**20,
    "rep": 1,
    "log_beta_wit_inf": 10
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm-poly", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("fold", {}),
    ("join", {}),
    ("batch", {}), 
])

uloop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm-poly", {}), 
    ("batch", {}), 
    ("usplit", {"d": ops_params["d"]}), 
    ("batch", {}), 
    ("urp", {}),
    ("fold", {}),
    ("batch", {}), 
])

ops = loop



ops = loop(1) + loop(2) * 8 + uloop(2)*3  + [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(8, 106, 106))
sim.ring.C = C


sim.execute(ops)
print(sim.ring)
sim.extract()
sim.show()

Limits of structure:  2048
coeff 2-norm
131072 524288
262144 262144
131072 131072
65536 65536
32768 32768
16384 16384
8192 8192
4096 4096
2048 2048
Ring parameters:
    conductor f: 512, degree phi: 256, modulus q: 2^50, beta_sis_2: 2^47.7000000000000
    SIS module rank n_sis: 7, target SIS security: 128, resulting SIS security: 129
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 100
    |R_q| = 1.56250 KB, |R_q^(n_sis)| = 10.9375 KB
 
Challenge set parameters:
    cardinality: 340282366920938463463374607431768211456
    forward ell_2 expansion factor gamma_2: 2^7
    inverse ell_2 expansion factor theta_2: 2^8
    forward ell_inf expansion factor gamma_inf: 2^7
    inverse ell_inf expansion factor theta_inf: 2^0
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |    1048576 |   1 |    ( 

In [21]:
# C1

ring_params = {
    "f": 512, # N = 256
    "log_beta_sis_2": 47.7,
    "log_q": 50,
}

rel_params = {
    "wdim": 2**22,
    "rep": 1,
    "log_beta_wit_inf": 10
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm-poly", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("fold", {}),
    ("join", {}),
    ("batch", {}), 
])

uloop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm-poly", {}), 
    ("batch", {}), 
    ("usplit", {"d": ops_params["d"]}), 
    ("batch", {}), 
    ("urp", {}),
    ("fold", {}),
    ("batch", {}), 
])

ops = loop



ops = loop(1) + loop(2) * 10 + uloop(2)*3  + [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(8, 106, 106))
sim.ring.C = C


sim.execute(ops)
print(sim.ring)
sim.extract()
sim.show()

Limits of structure:  2048
coeff 2-norm
524288 2097152
1048576 1048576
524288 524288
262144 262144
131072 131072
65536 65536
32768 32768
16384 16384
8192 8192
4096 4096
2048 2048
Ring parameters:
    conductor f: 512, degree phi: 256, modulus q: 2^50, beta_sis_2: 2^47.7000000000000
    SIS module rank n_sis: 7, target SIS security: 128, resulting SIS security: 129
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 100
    |R_q| = 1.56250 KB, |R_q^(n_sis)| = 10.9375 KB
 
Challenge set parameters:
    cardinality: 340282366920938463463374607431768211456
    forward ell_2 expansion factor gamma_2: 2^7
    inverse ell_2 expansion factor theta_2: 2^8
    forward ell_inf expansion factor gamma_inf: 2^7
    inverse ell_inf expansion factor theta_inf: 2^0
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init  

In [22]:
# A2

ring_params = {
    "f": 1024, # N = 512
    "log_beta_sis_2": 47.7,
    "log_q": 50,
}

rel_params = {
    "wdim": 2**17,
    "rep": 1,
    "log_beta_wit_inf": 10
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm-poly", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("fold", {}),
    ("join", {}),
    ("batch", {}), 
])

uloop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm-poly", {}), 
    ("batch", {}), 
    ("usplit", {"d": ops_params["d"]}), 
    ("batch", {}), 
    ("urp", {}),
    ("fold", {}),
    ("batch", {}), 
])

ops = loop



ops = loop(1) + loop(2) * 5 + uloop(2)*3  + [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(8, 106, 106))
sim.ring.C = C


sim.execute(ops)
print(sim.ring)
sim.extract()
sim.show()

Limits of structure:  2048
coeff 2-norm
16384 65536
32768 32768
16384 16384
8192 8192
4096 4096
2048 2048
Ring parameters:
    conductor f: 1024, degree phi: 512, modulus q: 2^50, beta_sis_2: 2^47.7000000000000
    SIS module rank n_sis: 4, target SIS security: 128, resulting SIS security: 150
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 100
    |R_q| = 3.12500 KB, |R_q^(n_sis)| = 12.5000 KB
 
Challenge set parameters:
    cardinality: 340282366920938463463374607431768211456
    forward ell_2 expansion factor gamma_2: 2^7
    inverse ell_2 expansion factor theta_2: 2^8
    forward ell_inf expansion factor gamma_inf: 2^7
    inverse ell_inf expansion factor theta_inf: 2^0
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |     131072 |   1 |    ( 23 | 23/  0 )        |     ( 10 | 23/  0 

In [23]:
# B2

ring_params = {
    "f": 1024, # N = 512
    "log_beta_sis_2": 47.7,
    "log_q": 50,
}

rel_params = {
    "wdim": 2**19,
    "rep": 1,
    "log_beta_wit_inf": 10
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm-poly", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("fold", {}),
    ("join", {}),
    ("batch", {}), 
])

uloop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm-poly", {}), 
    ("batch", {}), 
    ("usplit", {"d": ops_params["d"]}), 
    ("batch", {}), 
    ("urp", {}),
    ("fold", {}),
    ("batch", {}), 
])

ops = loop



ops = loop(1) + loop(2) * 7 + uloop(2)*3  + [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(8, 106, 106))
sim.ring.C = C


sim.execute(ops)
print(sim.ring)
sim.extract()
sim.show()

Limits of structure:  2048
coeff 2-norm
65536 262144
131072 131072
65536 65536
32768 32768
16384 16384
8192 8192
4096 4096
2048 2048
Ring parameters:
    conductor f: 1024, degree phi: 512, modulus q: 2^50, beta_sis_2: 2^47.7000000000000
    SIS module rank n_sis: 4, target SIS security: 128, resulting SIS security: 150
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 100
    |R_q| = 3.12500 KB, |R_q^(n_sis)| = 12.5000 KB
 
Challenge set parameters:
    cardinality: 340282366920938463463374607431768211456
    forward ell_2 expansion factor gamma_2: 2^7
    inverse ell_2 expansion factor theta_2: 2^8
    forward ell_inf expansion factor gamma_inf: 2^7
    inverse ell_inf expansion factor theta_inf: 2^0
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |     524288 |   1 |    ( 24 | 24/  0 ) 

In [24]:
# C2

ring_params = {
    "f": 1024, # N = 512
    "log_beta_sis_2": 47.7,
    "log_q": 50,
}

rel_params = {
    "wdim": 2**21,
    "rep": 1,
    "log_beta_wit_inf": 10
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm-poly", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("fold", {}),
    ("join", {}),
    ("batch", {}), 
])

uloop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm-poly", {}), 
    ("batch", {}), 
    ("usplit", {"d": ops_params["d"]}), 
    ("batch", {}), 
    ("urp", {}),
    ("fold", {}),
    ("batch", {}), 
])

ops = loop



ops = loop(1) + loop(2) * 9 + uloop(2)*3  + [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(8, 106, 106))
sim.ring.C = C


sim.execute(ops)
print(sim.ring)
sim.extract()
sim.show()

Limits of structure:  2048
coeff 2-norm
262144 1048576
524288 524288
262144 262144
131072 131072
65536 65536
32768 32768
16384 16384
8192 8192
4096 4096
2048 2048
Ring parameters:
    conductor f: 1024, degree phi: 512, modulus q: 2^50, beta_sis_2: 2^47.7000000000000
    SIS module rank n_sis: 4, target SIS security: 128, resulting SIS security: 150
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 100
    |R_q| = 3.12500 KB, |R_q^(n_sis)| = 12.5000 KB
 
Challenge set parameters:
    cardinality: 340282366920938463463374607431768211456
    forward ell_2 expansion factor gamma_2: 2^7
    inverse ell_2 expansion factor theta_2: 2^8
    forward ell_inf expansion factor gamma_inf: 2^7
    inverse ell_inf expansion factor theta_inf: 2^0
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |    20971

In [7]:
# FS 0 

C = ChallengeSet(cardinality = 2^128, gamma_2 = 128, theta_2 = 256, gamma_inf = 128, theta_inf = 1)

# params exactly as in LF+
ring_params = {
    "f": 256, # 2^ 7 deg
    "log_beta_sis_2": 43.6,
    "log_q": 50,
}

rel_params = {
    "wdim": 2**17,
    "rep": 4, #accumulator * 4
    "log_beta_wit_inf": 10
}


loop = lambda ell: ([
    ("fold-ext", { "extra_rels": 4 }), # now we add extra relation + send crossterms. I decided to fold 4 relations so we have power-of-two columns  
    ("norm-poly", {}), # only for the accumulator 
    ("batch", {}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("fold", {}),
    ("batch", {}), 
    ("join", {}),
    ("batch", {}),  # whis is different batch but comunication-wise it's the same
    ("bdecomp", {"ell":ell }), 
])

ops = loop



ops = loop(2) + [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(8, 128, 8))
sim.ring.C = C


sim.execute(ops)
print(sim.ring)
sim.extract()
sim.show()

Limits of structure:  2048
coeff 2-norm
131072 131072
Ring parameters:
    conductor f: 256, degree phi: 128, modulus q: 2^50, beta_sis_2: 2^43.6000000000000
    SIS module rank n_sis: 12, target SIS security: 128, resulting SIS security: 132
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 100
    |R_q| = 800.000 B, |R_q^(n_sis)| = 9.37500 KB
 
Challenge set parameters:
    cardinality: 340282366920938463463374607431768211456
    forward ell_2 expansion factor gamma_2: 2^7
    inverse ell_2 expansion factor theta_2: 2^8
    forward ell_inf expansion factor gamma_inf: 2^7
    inverse ell_inf expansion factor theta_inf: 2^0
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |     131072 |   4 |    ( 22 | 22/  0 )        |     ( 10 | 22/  0 )         | 88.0000 MB |      (0.000000 B | 0.000000 

In [10]:
# FS 1 

C = ChallengeSet(cardinality = 2^128, gamma_2 = 128, theta_2 = 256, gamma_inf = 128, theta_inf = 1)

# params exactly as in LF+
ring_params = {
    "f": 256, # 2^ 7 deg
    "log_beta_sis_2": 44.6,
    "log_q": 50,
}

rel_params = {
    "wdim": 2**19,
    "rep": 4, #accumulator * 4
    "log_beta_wit_inf": 10
}


loop = lambda ell: ([
    ("fold-ext", { "extra_rels": 4 }), # now we add extra relation + send crossterms. I decided to fold 4 relations so we have power-of-two columns  
    ("norm-poly", {}), # only for the accumulator 
    ("batch", {}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("fold", {}),
    ("batch", {}), 
    ("join", {}),
    ("batch", {}),  # whis is different batch but comunication-wise it's the same
    ("bdecomp", {"ell":ell }), 
])

ops = loop



ops = loop(2) + [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(8, 128, 8))
sim.ring.C = C


sim.execute(ops)
print(sim.ring)
sim.extract()
sim.show()

Limits of structure:  2048
coeff 2-norm
524288 524288
Ring parameters:
    conductor f: 256, degree phi: 128, modulus q: 2^50, beta_sis_2: 2^44.6000000000000
    SIS module rank n_sis: 13, target SIS security: 128, resulting SIS security: 138
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 100
    |R_q| = 800.000 B, |R_q^(n_sis)| = 10.1562 KB
 
Challenge set parameters:
    cardinality: 340282366920938463463374607431768211456
    forward ell_2 expansion factor gamma_2: 2^7
    inverse ell_2 expansion factor theta_2: 2^8
    forward ell_inf expansion factor gamma_inf: 2^7
    inverse ell_inf expansion factor theta_inf: 2^0
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |     524288 |   4 |    ( 23 | 23/  0 )        |     ( 10 | 23/  0 )         | 352.000 MB |      (0.000000 B | 0.000000 

In [13]:
# FS 2

C = ChallengeSet(cardinality = 2^128, gamma_2 = 128, theta_2 = 256, gamma_inf = 128, theta_inf = 1)

# params exactly as in LF+
ring_params = {
    "f": 256, # 2^ 7 deg
    "log_beta_sis_2": 45.6,
    "log_q": 50,
}

rel_params = {
    "wdim": 2**21,
    "rep": 4, #accumulator * 4
    "log_beta_wit_inf": 10
}


loop = lambda ell: ([
    ("fold-ext", { "extra_rels": 4 }), # now we add extra relation + send crossterms. I decided to fold 4 relations so we have power-of-two columns  
    ("norm-poly", {}), # only for the accumulator 
    ("batch", {}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("fold", {}),
    ("batch", {}), 
    ("join", {}),
    ("batch", {}),  # whis is different batch but comunication-wise it's the same
    ("bdecomp", {"ell":ell }), 
])

ops = loop



ops = loop(2) + [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(8, 128, 8))
sim.ring.C = C


sim.execute(ops)
print(sim.ring)
sim.extract()
sim.show()

Limits of structure:  2048
coeff 2-norm
2097152 2097152
Ring parameters:
    conductor f: 256, degree phi: 128, modulus q: 2^50, beta_sis_2: 2^45.6000000000000
    SIS module rank n_sis: 13, target SIS security: 128, resulting SIS security: 131
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 100
    |R_q| = 800.000 B, |R_q^(n_sis)| = 10.1562 KB
 
Challenge set parameters:
    cardinality: 340282366920938463463374607431768211456
    forward ell_2 expansion factor gamma_2: 2^7
    inverse ell_2 expansion factor theta_2: 2^8
    forward ell_inf expansion factor gamma_inf: 2^7
    inverse ell_inf expansion factor theta_inf: 2^0
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |    2097152 |   4 |    ( 24 | 24/  0 )        |     ( 10 | 24/  0 )         | 1408.00 MB |      (0.000000 B | 0.00000